In [2]:
import pickle
import itertools

with open('Cliff_index_gateset2.pkl', 'rb') as f:
    a = pickle.load(f)

a=list(itertools.chain.from_iterable(a))
b=[a.count(i) for i in range(15)]
print(b)


[16192, 11846, 15055, 13880, 10476, 10481, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
[1, 9601, 8129, 3103, 2296, 0, 0, 9285, 8118, 2974, 3058, 0, 0, 10523, 9788]

In [53]:
from pyquil.simulation.matrices import I, X, Y, Z
import numpy as np
from forest.benchmarking.operator_tools import kraus2pauli_liouville
from print_large import _print_big_matrix
import pickle
#gateset1 [1, 9601, 8129, 3103, 2296, 0, 0, 9285, 8118, 2974, 3058, 0, 0, 10523, 9788]
#gateset2 [16192, 11846, 15055, 13880, 10476, 10481]
###rb_seq_deompose
with open('rb_seq_decompose_gateset1.pkl', 'rb') as f:
    rb_seq_decompose = pickle.load(f)
###primitive gate PTM
with open('P_PTM_gateset1.pkl', 'rb') as f:
    P_PTM = pickle.load(f)
#### 2-qubit pauli basis
pauli_basis=[np.asarray(np.kron(i,j)) for i in [I, X, Y, Z] for j in [I, X, Y, Z]]
#### 2-qubit |0> and |1>
e_state=np.kron(np.asarray([[0+0j,0],[0,1]]),np.asarray([[0,0],[0,1]]))
g_state=np.kron(np.asarray([[1+0j,0],[0,0]]),np.asarray([[1,0],[0,0]]))
####Density matrix to PTM
def densityop_01_2_pauli(density_op):
    #assert density_op.ndim !=4,'wrong dim'
    A=np.asarray([np.trace(density_op@i_den) for i_den in pauli_basis]).reshape(density_op.size,1)
    return A
def densityop_pauli_2_01(density_op):
    #assert density_op.shape == (len(density_op),1),'wrong dim'
    A_den=np.asarray([density_op[i_den]*pauli_basis[i_den] for i_den in range(16)])
    B_den=np.zeros((4,4))
    for i_den in A_den:
        B_den=B_den+i_den
    return B_den/4

def depolarizing_PTM(p):
    A=[np.kron(i_depo,j_depo) for i_depo in [I,X,Y,Z] for j_depo in [I,X,Y,Z]]
    A[0] = A[0]*np.sqrt(1-15*p/16)
    for i_depo in range(1,len(A)):
        A[i_depo]=A[i_depo]*np.sqrt(p/16)
#########kraus to PTM        
    return A
def dephaseing_PTM(p):
    A=[np.kron(i_deph,j_deph) for i_deph in [I,Z] for j_deph in [I,Z]]
    for i_deph in [0]:
        A[i_deph] = A[i_deph]*np.sqrt(p+(1-p)/4)
    for i_deph in [1,2,3]:
        A[i_deph] = A[i_deph]*np.sqrt((1-p)/4)
#########kraus to PTM 
    return kraus2pauli_liouville(A)
state=np.zeros((4,4),dtype=complex)
c=dephaseing_PTM(0)
_print_big_matrix(c)
'''for i in c:
    state += i @e_state@ np.conj(i).T
for i in range(4):
    print(state[i][i])'''
 

1.0  .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .  1.0  .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .  1.0  .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .  
 .   .   .   .   .   .   

'for i in c:\n    state += i @e_state@ np.conj(i).T\nfor i in range(4):\n    print(state[i][i])'